# Project title: Diabet Prediction

## Authors: Denys Herasymuk & Yaroslav Morozevych

**Variant**: the remainder of the division = 3

## Contents of This Notebook

Click on the section and go to this cell immediately.

* [Section 1. Explore Data](#section_1)
* [Section 2. Identifying Stationarity](#section_2)
* [Section 3. Nonstationary-to-Stationary Transformations](#section_3)
* [Section 4. Correlation analysis](#section_4)
* [Section 5. Feature generation and validation of DL models](#section_5)
* [Section 6. Fbprophet and Nbeats models](#section_6)
* [Section 7. Predict on 12 months out of dataframe](#section_7)


When you use Run All button with this notebook, you should wait approx. 10-15 mins to get output of all cells.

**How to run this notebook**

* Create a new conda env with python 3.7
* Run `jupyter notebook` in your new env via terminal (without installing any packages now)
* Based on (this link)[https://stackoverflow.com/questions/61353951/no-module-named-fbprophet]. Run these two cells in your jupyter:
```
!conda install -c conda-forge fbprophet -y
!pip install --upgrade plotly
```

* In terminal run  -- `pip install -r requirements.txt`
* In any case, a useful command -- `conda create --clone py35 --name py35-2` from here -- https://docs.conda.io/projects/conda/en/4.6.0/_downloads/52a95608c49671267e40c689e0bc00ca/conda-cheatsheet.pdf

* How to install pytorch -- https://pytorch.org/get-started/locally/
* In such a way I installed it on Ubuntu
`pip3 install torch==1.10.0+cpu torchvision==0.11.1+cpu torchaudio==0.10.0+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html`

* Nbeats installing -- `pip3 install nbeats-pytorch`

## General Configuration

In [1]:
import os
import sys
import math
import sklearn
import itertools
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels as ss
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

# alt.data_transformers.disable_max_rows()
# alt.renderers.enable('html')

plt.style.use('mpl20')
matplotlib.rcParams['figure.dpi'] = 100
matplotlib.rcParams['figure.figsize'] = 15, 5

# import warnings
# warnings.filterwarnings('ignore')

## Python & Library Versions

In [3]:
versions = ( ("matplotlib", matplotlib.__version__),
             ("numpy", np.__version__),
             ("pandas", pd.__version__),
             ("statsmodels", ss.__version__),
             ("seaborn", sns.__version__),
             ("sklearn", sklearn.__version__),
             # ("keras", keras.__version__),
             # ("xgboost", xgboost.__version__),
             )

print(sys.version, "\n")
print("library" + " " * 4 + "version")
print("-" * 18)

for tup1, tup2 in versions:
    print("{:11} {}".format(tup1, tup2))

3.8.12 (default, Oct 12 2021, 13:49:34) 
[GCC 7.5.0] 

library    version
------------------
matplotlib  3.5.1
numpy       1.19.2
pandas      1.3.5
statsmodels 0.13.1
seaborn     0.11.2
sklearn     1.0.1


<a id='section_1'></a>

## Section 1. Explore Data

In [4]:
diabetes_df = pd.read_csv(os.path.join(".", "data", "diabetes.csv"))

In [5]:
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
diabetes_df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
diabetes_df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [11]:
total_n_rows = len(diabetes_df)
print('Total number of occurrences: ', total_n_rows)
print('Number of diabet occurrences: ', round(len(diabetes_df[diabetes_df.Outcome == 1]) / total_n_rows, 2))
print('Number of non-diabet occurrences: ', round(len(diabetes_df[diabetes_df.Outcome == 0]) / total_n_rows, 2))

Total number of occurrences:  768
Number of diabet occurrences:  0.35
Number of non-diabet occurrences:  0.65
